In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-07 14:32:47


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}
Segs[2017] = Segs[2016]


UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-09-07 14:32:48


In [60]:
# Map M crashes to P Direction
df = hsmpy3.common.FCtoDF(Routes[2016],selectedFields=['ROUTE_ID','Shape'])
Route_Dict = {year:df for year in range(2012,2018)}
for year in range(2012,2018):
    if not 'RID_P' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'RID_P','Text',50)
    if not 'MP_P' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'MP_P','Double')
    uc = arcpy.UpdateCursor(CrashData[year])
    for r in uc:
        if len(r.getValue('RID').split('-'))>2:
            if r.getValue('RID').split('-')[2] == 'M':
                rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==hsmpy3.nm.OppositeDirID(r.getValue('RID'))]
                pnt,disalongr,offset,side = rdf['Shape'].item().queryPointAndDistance(r.getValue('Shape'))
                mp = pnt.firstPoint.M
                r.setValue('RID_P',str(hsmpy3.nm.OppositeDirID(r.getValue('RID'))))
                r.setValue('MP_P',float(mp))
                uc.updateRow(r)
            else:
                r.setValue('RID_P',str(r.getValue('RID')))
                r.setValue('MP_P',float(r.getValue('MP')))
                uc.updateRow(r)
        else:
            r.setValue('RID_P',str(r.getValue('RID')))
            r.setValue('MP_P',float(r.getValue('MP')))
            uc.updateRow(r)
    del uc
    del r
    print(year)

2012
2013
2014
2015
2016
2017


In [7]:
#Create Crash Assignment Tables
Diss_Fields = ['F_SYSTEM','OWNERSHIP','URBAN_CODE','SegID','FType','Int_ID','IFType']
for year in range(2012,2017):
    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(year),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[year],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    for f in [F.name for F in arcpy.ListFields(Tab)]:
        if f in Diss_Fields:
            arcpy.management.DeleteField(Tab,f)
    uc = arcpy.UpdateCursor(Tab)
    for r in uc:
        r.setValue('RID',str(r.getValue('RID_P')))
        r.setValue('MP',str(r.getValue('MP_P')))
        uc.updateRow(r)
    del uc
    del r
    arcpy.management.DeleteField(Tab,'RID_P')
    arcpy.management.DeleteField(Tab,'MP_P')
    Diss = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegAtt_Int_Diss',Extension='',appendix=str(year))
    arcpy.DissolveRouteEvents_lr (in_events = Project_GDB+'\\SegAtt_Int_'+str(year), 
                                  in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_field = Diss_Fields, 
                                  out_table = Diss, 
                                  out_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_type = 'DISSOLVE', 
                                  build_index = 'INDEX')  

    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(year),'')
    arcpy.OverlayRouteEvents_lr(in_table = Tab, 
                                in_event_properties = ' '.join(['RID','POINT','MP']),
                                overlay_table = Diss, 
                                overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),  
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    print(year)

2012
2013
2014
2015
2016


In [8]:
#Create Crash Assignment Tables 2017
Diss_Fields = ['F_SYSTEM','OWNERSHIP','URBAN_CODE','SegID','FType','Int_ID','IFType']
if 1<2:

    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(2017),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[2017],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    for f in [F.name for F in arcpy.ListFields(Tab)]:
        if f in Diss_Fields:
            arcpy.management.DeleteField(Tab,f)

    uc = arcpy.UpdateCursor(Tab)
    for r in uc:
        r.setValue('RID',str(r.getValue('RID_P')))
        r.setValue('MP',str(r.getValue('MP_P')))
        uc.updateRow(r)
    del uc
    del r
    arcpy.management.DeleteField(Tab,'RID_P')
    arcpy.management.DeleteField(Tab,'MP_P')
            
    Diss = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegAtt_Int_Diss',Extension='',appendix=str(2016))
    arcpy.DissolveRouteEvents_lr (in_events = Project_GDB+'\\SegAtt_Int_'+str(2016), 
                                  in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_field = Diss_Fields, 
                                  out_table = Diss, 
                                  out_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_type = 'DISSOLVE', 
                                  build_index = 'INDEX')  

    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(2017),'')
    arcpy.OverlayRouteEvents_lr(in_table = Tab, 
                                in_event_properties = ' '.join(['RID','POINT','MP']),
                                overlay_table = Diss, 
                                overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),  
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    print(2017)

2017


In [9]:
# Map Crash Assignments to Crash Files
for year in range(2012,2018):
    df = hsmpy3.common.FCtoDF(Project_GDB+'\\Crash_Assignment_'+str(year))
    if not 'F_SYSTEM' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'F_SYSTEM','Single')
    if 'FACILITY_TYPE' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.DeleteField(CrashData[year],'FACILITY_TYPE')
    if not 'URBAN_CODE' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'URBAN_CODE','Single')
    if not 'OWNERSHIP' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'OWNERSHIP','Single')
    if not 'FType' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'FType','Text')
    if not 'IFType' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'IFType','Text')
    if not 'Int_ID' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'Int_ID','Long')
    if not 'SegID' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'SegID','Text',50)
    uc = arcpy.UpdateCursor(CrashData[year])
    for r in uc:
        ucr = r.getValue('UCRnum')
        cdf = df[df['UCRnum'] == ucr]
        if cdf.shape[0]==1:
            if cdf['F_SYSTEM'].item() !='':
                r.setValue('F_SYSTEM'     ,int(cdf['F_SYSTEM'].item()))
            #if cdf['FACILITY_TYPE'].item() !='':
            #    r.setValue('FACILITY_TYPE',int(cdf['FACILITY_TYPE'].item()))
            if cdf['URBAN_CODE'].item() !='':
                r.setValue('URBAN_CODE',int(cdf['URBAN_CODE'].item()))
            if cdf['OWNERSHIP'].item() !='':
                r.setValue('OWNERSHIP',int(cdf['OWNERSHIP'].item()))
            if cdf['FType'].item() !='':
                r.setValue('FType',cdf['FType'].item())
            if cdf['IFType'].item() !='':
                r.setValue('IFType',cdf['IFType'].item())
            if cdf['Int_ID'].item() !='':
                r.setValue('Int_ID',cdf['Int_ID'].item())
            if cdf['SegID'].item() !='':
                r.setValue('SegID',cdf['SegID'].item())
            uc.updateRow(r)
    del uc
    del r
    print(year)

2012
2013
2014
2015
2016
2017


In [12]:
for year in range(2012,2017):
    arcpy.management.Delete(Project_GDB + "\\R_Route_Seg_"+str(year))
    arcpy.management.CreateRelationshipClass(
        Routes[2016], 
        Segs[year], 
        Project_GDB + "\\R_Route_Seg_"+str(year), 
        "SIMPLE", 
        "Seg_" + str(year), 
        "Routes", 
        "FORWARD", 
        "ONE_TO_MANY", 
        "NONE", 
        'ROUTE_ID', 
        'ROUTE_ID'
    )
    arcpy.management.Delete(Project_GDB + "\\R_Route_Crash_"+str(year))
    arcpy.management.CreateRelationshipClass(
        Routes[2016], 
        CrashData[year], 
        Project_GDB + "\\R_Route_Crash_"+str(year), 
        "SIMPLE", 
        "Crash_" + str(year), 
        "Routes", 
        "FORWARD", 
        "ONE_TO_MANY", 
        "NONE", 
        'ROUTE_ID', 
        'RID_P'
    )
    arcpy.management.Delete(Project_GDB + "\\R_Seg_Crash_"+str(year))
    arcpy.management.CreateRelationshipClass(
        Segs[year], 
        CrashData[year], 
        Project_GDB + "\\R_Seg_Crash_"+str(year), 
        "SIMPLE", 
        "Crash_" + str(year), 
        "Seg_" + str(year), 
        "FORWARD", 
        "ONE_TO_MANY", 
        "NONE", 
        'SegID', 
        'SegID'
    )
    arcpy.management.Delete(Project_GDB + "\\R_Int_Crash_"+str(year))
    arcpy.management.CreateRelationshipClass(
        Project_GDB + '\\Int_' + str(year), 
        CrashData[year], 
        Project_GDB + "\\R_Int_Crash_"+str(year), 
        "SIMPLE", 
        "Crash_" + str(year), 
        "Int_" + str(year), 
        "FORWARD", 
        "ONE_TO_MANY", 
        "NONE", 
        'Int_ID', 
        'Int_ID'
    )
    print(year)

2012
2013
2014
2015
2016
